
# NaviGator Toolkit File Upload API Demo

## NOTE: This only works with cloud models, not locally hosted models.

> As of early 2026, the NaviGator Toolkit API supports file uploads for certain cloud models. This allows you to provide additional context or data to the model when making requests. **Locally hosted models do not currently support file uploads**, so this feature is only available when using cloud-based models through the NaviGator API.


----

This notebook demonstrates uploading a file using the NaviGator Toolkit API ([see here for more information](https://it.ufl.edu/ai/navigator-toolkit/)). File uploads only work with certain cloud models; locally hosted models do not currently support uploads.

You will need a NaviGator API key. Store the key in a `.json` file with the following format:

    {
      "OPENAI_API_KEY" : "Put your key here in the quotes",
      "base_url" : "https://api.ai.it.ufl.edu/"
    }

We recommend storing the file in your home directory to reduce the chance of accidentally committing it to a git repo. Anyone with your API key can use NaviGator as you! 

In [1]:
import openai
import os
import json
import base64


## Load `.json` file with your key and API endpoint URL

In [2]:
# Set the path to your jsnkey file
key_file = '/home/magitz/navigator_api_keys.json'


# Load the JSON file
with open(key_file, 'r') as file:
    data = json.load(file)

# Extract the values
OPENAI_API_KEY = data.get('OPENAI_API_KEY')
base_url = data.get('base_url')

# Set the environment variable
os.environ['TOOLKIT_API_KEY'] = OPENAI_API_KEY


## Test connectivity and get model list

The reply should list the models available with your API key. An example (truncated) output is:

    SyncPage[Model](data=[Model(id='llama-3.1-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='sfr-embedding-mistral', created=1677610602, object='model', owned_by='openai'),...)], object='list')

In [3]:
# Check list of available models
client = openai.OpenAI(
    api_key=os.environ.get("TOOLKIT_API_KEY"),
    base_url=base_url
)

if not hasattr(client, "responses"):
    raise RuntimeError(
        "OpenAI SDK is missing the responses API. "
        "Upgrade to a newer openai package (e.g., pip install -U openai) "
        "or use a kernel/environment that includes responses."
    )

response = client.models.list()
 
print(response)

SyncPage[Model](data=[Model(id='llama-3.1-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.1-8b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.1-nemotron-nano-8B-v1', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.3-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='mistral-7b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='mistral-small-3.1', created=1677610602, object='model', owned_by='openai'), Model(id='codestral-22b', created=1677610602, object='model', owned_by='openai'), Model(id='gemma-3-27b-it', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-oss-20b', created=1677610602, object='model', owned_by='openai'), Model(id='gpt-oss-120b', created=1677610602, object='model', owned_by='openai'), Model(id='granite-3.3-8b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='sfr-em

In [4]:
# Print available models in better format
for model in response:
    print(model.id)

llama-3.1-70b-instruct
llama-3.1-8b-instruct
llama-3.1-nemotron-nano-8B-v1
llama-3.3-70b-instruct
mistral-7b-instruct
mistral-small-3.1
codestral-22b
gemma-3-27b-it
gpt-oss-20b
gpt-oss-120b
granite-3.3-8b-instruct
sfr-embedding-mistral
nomic-embed-text-v1.5
flux.1-dev
flux.1-schnell
whisper-large-v3
kokoro
o1
o3
o3-mini
o3-mini-high
o3-mini-medium
o4-mini
o4-mini-high
o4-mini-medium
gpt-4.1
gpt-4.1-mini
gpt-4.1-nano
gpt-4o
gpt-4o-mini
gpt-5
gpt-5-mini
gpt-5-nano
gpt-5.1
gpt-5.1-codex
gpt-5.1-codex-mini
gpt-5.2
gemini-2.0-flash
gemini-2.5-flash
gemini-2.5-pro
claude-3.5-haiku
claude-3.5-sonnet
claude-3.5-sonnet-v2
claude-3.7-sonnet
claude-3.7-sonnet-thinking
claude-3-haiku
claude-3-opus
claude-4-sonnet
claude-4.5-sonnet
claude-4.5-sonnet-thinking
nova-lite
nova-micro
nova-pro
mistral-large
mistral-large-2
mistral-small


## Helper function to convert LiteLLM filename to a usable filename

LiteLLM, the API endpoint, encodes the uploaded file in base64 with the model name as part of the string. We need to decode that and strip out just the file name to send to a cloud model. Otherwise, we exceed the name length limits—at least with the Azure-hosted OpenAI models. 

In [5]:
def decode_base64(encoded: str) -> bytes:
    """Decode a base64-encoded string into raw bytes."""
    # Add padding if missing
    missing_padding = len(encoded) % 4
    if missing_padding:
        encoded += "=" * (4 - missing_padding)
    return base64.b64decode(encoded)
    
def get_file_name(encoded_name, debug=False):
    """Take a base64-encoded file path and return the file name."""

    litellmFileBytes = decode_base64(encoded_name.split("-")[1])
    if debug:
        print(f"litellmFileBytes: {litellmFileBytes}")
    
    litellmFile = litellmFileBytes.decode('utf-8')
    if debug:
        print(f"litellmFile: {litellmFile}")
    
    file_id = litellmFile.split(":")[1].split(";")[0]
    if debug:
        print(f"Usable file_id is: {file_id}")

    return file_id


## Load a file

In this example, we use the NVIDIA Medium-Range Weather forecast model paper available here: [https://d1qx31qr3h6wln.cloudfront.net/publications/atlas-paper.pdf](https://d1qx31qr3h6wln.cloudfront.net/publications/atlas-paper.pdf). This paper is in the `data` folder of the repository.


In [6]:
# Define the file to upload 
file = "data/Kossaifi_et_el_2026_NVIDIA_Medium-Range_Forecast.pdf"

# Define the purpose and model
purpose = "assistants"
model = "gpt-4o"


# Upload a file to the API
print(f"Uploading file: {file}...")

try:
    uploaded_file = client.files.create(
        file = open(file, "rb"),
        purpose = purpose,
        extra_body = {
            "model": model
        },
    )
except Exception as e:
    print(f"Error: Uploading file failed.")
    print(f"Error: Exception is: {e}")
    exit(1)

print(f"✅ Uploaded file (encoded name): {uploaded_file.id}")
print(f"Decoding file name...")
file_id = get_file_name(uploaded_file.id, debug=False)
print(f"✅ Usable file_id is: {file_id}")

Uploading file: data/Kossaifi_et_el_2026_NVIDIA_Medium-Range_Forecast.pdf...
✅ Uploaded file (encoded name): file-bGl0ZWxsbTphc3Npc3RhbnQtODR6OTR2QVplNnd4b0J1VjROZDRrdTttb2RlbCxncHQtNG8
Decoding file name...
✅ Usable file_id is: assistant-84z94vAZe6wxoBuV4Nd4ku


## Create a summary of the file

In [7]:
# Deine the prompt
prompt = "Summarize the key findings of the document."

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that provides a variety of AI file services."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "input_text",
                "text": prompt
            },
            {
                "type": "input_file",
                "file_id": file_id
            }
        ]
    }
]

try:
    print("Attempting to create a response...")
    response = client.responses.create(
        model = model,
        input = messages
    )

    response_id = response.id

    try:
        retrieved_response = client.responses.retrieve(response_id)
        print(f"Response is:\n {retrieved_response.output_text}\n")
    except Exception as e:
        print(f"Failed to retrieve response: {e}")

except Exception as e:
    print(f"An expection occured while trying to create a chat session: {e}")




Attempting to create a response...
Response is:
 The document introduces a new framework called ATLAS for medium-range probabilistic weather forecasting. Key findings include:

1. **Unified Approach over Complexity**:
   - The study demonstrates that state-of-the-art probabilistic skill in weather forecasting does not require complex bespoke architectures or specialized training methods.
   - ATLAS leverages a streamlined, scalable design with standard transformer architectures and a history-conditioned local projector, proving effective in capturing high-resolution physics.

2. **Method-Agnostic Robustness**:
   - The framework support various probabilistic methods such as stochastic interpolants, diffusion models, and CRPS-based ensemble training, performing robustly across different estimation strategies.

3. **Performance**:
   - ATLAS outperforms the European Centre for Medium-Range Weather Forecasts’ Integrated Forecasting System (IFS) and the deep learning-based model GenCast on